In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from amplpy import AMPL

In [ ]:
AMPL_FOLDER = r'C:\Users\crhac\Desktop\CATTSSP\AMPL\data'
AMPL_MOD = r'C:\Users\crhac\Desktop\CATTSSP\AMPL\modelo.mod'

In [ ]:
def ejecutar_instancia(dat_path, time_limit=120):
    ampl = AMPL()
    ampl.read(AMPL_MOD)
    ampl.read_data(dat_path)
    
    ampl.set_option('solver', 'highs')
    ampl.set_option('highs_options', f'time_limit={time_limit}')
    
    inicio = time.time()
    ampl.solve()
    fin = time.time()

    x = ampl.get_variable('x')
    arcos = [(i, j) for i, j, val in x.get_values().to_list() if val > 0.5]

    obj = ampl.get_objective('TotalCost')
    costo = obj.value()

    return costo, fin - inicio, arcos

In [ ]:
def graficar_solucion(arcos, dimension, titulo):
    np.random.seed(0)
    coords = np.random.rand(dimension, 2)

    plt.figure(figsize=(6, 6))
    plt.scatter(coords[:, 0], coords[:, 1], color='blue')

    for idx, (x, y) in enumerate(coords):
        plt.text(x, y, str(idx), fontsize=9, ha='right')

    for i, j in arcos:
        plt.plot([coords[i, 0], coords[j, 0]], [coords[i, 1], coords[j, 1]], 'r-')

    plt.title(titulo)
    plt.show()

In [ ]:
archivo = "data.dat"
dat_path = os.path.join(AMPL_FOLDER, archivo)

costo, tiempo, arcos = ejecutar_instancia(dat_path, time_limit=120)

print(f"\nInstancia: {archivo}")
print(f"Costo total: {costo}")
print(f"Tiempo de ejecución: {tiempo:.4f} segundos")
print(f"Arcos seleccionados: {arcos}")

graficar_solucion(arcos, len(set(i for i, j in arcos) | set(j for i, j in arcos)), f"{archivo} - Costo: {costo}")